In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1020155,2021-02-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1020156,2021-02-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1020157,2021-02-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1020158,2021-02-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27699,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27701,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27703,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27705,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27707,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
632157,2021-02-21,Arkansas,Arkansas,5001,1942,38.00,5000,Arkansas,AR,Arkansas,State,3017804
632159,2021-02-22,Arkansas,Arkansas,5001,1944,38.00,5000,Arkansas,AR,Arkansas,State,3017804
632161,2021-02-23,Arkansas,Arkansas,5001,1949,38.00,5000,Arkansas,AR,Arkansas,State,3017804
632163,2021-02-24,Arkansas,Arkansas,5001,1963,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1020155,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1020156,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1020157,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1020158,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-25') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
401,2021-02-25,Snohomish,Washington,30754,538.00,53061,WA,County,822083,3740.98,65.44
800,2021-02-25,Cook,Illinois,472206,9872.00,17031,IL,County,5150233,9168.63,191.68
1198,2021-02-25,Orange,California,260348,3889.00,6059,CA,County,3175692,8198.15,122.46
1595,2021-02-25,Maricopa,Arizona,508456,9005.00,4013,AZ,County,4485414,11335.77,200.76
1992,2021-02-25,Los Angeles,California,1188101,21102.00,6037,CA,County,10039107,11834.73,210.20
...,...,...,...,...,...,...,...,...,...,...,...
1019739,2021-02-25,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1019875,2021-02-25,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1019980,2021-02-25,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1020081,2021-02-25,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
401,2021-02-25,Snohomish,Washington,30754,538.00,53061,WA,County,822083,3740.98,65.44,65.44,3740.98
800,2021-02-25,Cook,Illinois,472206,9872.00,17031,IL,County,5150233,9168.63,191.68,191.68,9168.63
1198,2021-02-25,Orange,California,260348,3889.00,6059,CA,County,3175692,8198.15,122.46,122.46,8198.15
1595,2021-02-25,Maricopa,Arizona,508456,9005.00,4013,AZ,County,4485414,11335.77,200.76,200.76,11335.77
1992,2021-02-25,Los Angeles,California,1188101,21102.00,6037,CA,County,10039107,11834.73,210.20,210.20,11834.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019739,2021-02-25,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1019875,2021-02-25,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1019980,2021-02-25,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1020081,2021-02-25,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
719081,2021-02-25,Clarke,Alabama,3429,49.00,1025,AL,County,23622,14516.13,207.43,207.43,14516.13,1
660886,2021-02-25,Hale,Alabama,2088,68.00,1065,AL,County,14651,14251.59,464.13,464.13,14251.59,2
517581,2021-02-25,Lowndes,Alabama,1306,48.00,1085,AL,County,9726,13427.93,493.52,493.52,13427.93,3
459113,2021-02-25,Etowah,Alabama,13143,319.00,1055,AL,County,102268,12851.53,311.93,311.93,12851.53,4
385476,2021-02-25,Franklin,Alabama,3978,76.00,1059,AL,County,31362,12684.14,242.33,242.33,12684.14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970188,2021-02-25,Platte,Wyoming,587,11.00,56031,WY,County,8393,6993.92,131.06,131.06,6993.92,19
631163,2021-02-25,Sublette,Wyoming,673,7.00,56035,WY,County,9831,6845.69,71.20,71.20,6845.69,20
871269,2021-02-25,Niobrara,Wyoming,148,2.00,56027,WY,County,2356,6281.83,84.89,84.89,6281.83,21
783433,2021-02-25,Lincoln,Wyoming,1221,11.00,56023,WY,County,19830,6157.34,55.47,55.47,6157.34,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
517581,2021-02-25,Lowndes,Alabama,1306,48.00,1085,AL,County,9726,13427.93,493.52,493.52,13427.93,3,1
660886,2021-02-25,Hale,Alabama,2088,68.00,1065,AL,County,14651,14251.59,464.13,464.13,14251.59,2,2
457761,2021-02-25,Clay,Alabama,1433,54.00,1027,AL,County,13235,10827.35,408.01,408.01,10827.35,21,3
220694,2021-02-25,Walker,Alabama,6504,251.00,1127,AL,County,63521,10239.13,395.14,395.14,10239.13,32,4
558018,2021-02-25,Greene,Alabama,870,32.00,1063,AL,County,8111,10726.17,394.53,394.53,10726.17,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556673,2021-02-25,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,17,19
764022,2021-02-25,Uinta,Wyoming,2048,12.00,56041,WY,County,20226,10125.58,59.33,59.33,10125.58,4,20
783433,2021-02-25,Lincoln,Wyoming,1221,11.00,56023,WY,County,19830,6157.34,55.47,55.47,6157.34,22,21
219662,2021-02-25,Teton,Wyoming,3342,9.00,56039,WY,County,23464,14243.10,38.36,38.36,14243.10,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3740.98,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3740.98,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3740.98,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3740.98,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3740.98,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017747,2021-02-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1017748,2021-02-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1017749,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1017750,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
716343,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,207.43,14516.13,47,1,2.00
716344,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,207.43,14516.13,47,1,0.00
716345,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,207.43,14516.13,47,1,1.00
716346,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,207.43,14516.13,47,1,4.00
716347,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,207.43,14516.13,47,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796437,2021-02-21,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
796438,2021-02-22,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
796439,2021-02-23,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
796440,2021-02-24,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
515507,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13427.93,1,3,1.00,0.00
515508,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13427.93,1,3,0.00,0.00
515509,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13427.93,1,3,0.00,0.00
515510,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13427.93,1,3,0.00,0.00
515511,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,493.52,13427.93,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513817,2021-02-21,Albany,Wyoming,3887,11.00,56001,WY,County,38880,9997.43,28.29,28.29,10015.43,23,6,7.00,0.00
513818,2021-02-22,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10015.43,23,6,6.00,0.00
513819,2021-02-23,Albany,Wyoming,3892,11.00,56001,WY,County,38880,10010.29,28.29,28.29,10015.43,23,6,-1.00,0.00
513820,2021-02-24,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10015.43,23,6,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-25') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
263318,2021-02-25,Imperial,California,26843,585.00,6025,CA,County,181215,14812.79,322.82,322.82,14812.79,1,2,3.00,1.00
1992,2021-02-25,Los Angeles,California,1188101,21102.00,6037,CA,County,10039107,11834.73,210.20,210.20,11834.73,2,5,2027.00,115.00
460417,2021-02-25,Inyo,California,1253,36.00,6027,CA,County,18039,6946.06,199.57,199.57,6946.06,3,28,3.00,1.00
58865,2021-02-25,Stanislaus,California,55775,944.00,6099,CA,County,550660,10128.75,171.43,171.43,10128.75,4,10,150.00,3.00
72942,2021-02-25,Tulare,California,47784,746.00,6107,CA,County,466195,10249.79,160.02,160.02,10249.79,5,9,54.00,7.00
27342,2021-02-25,Riverside,California,289185,3749.00,6065,CA,County,2470546,11705.31,151.75,151.75,11705.31,6,6,225.00,42.00
557960,2021-02-25,Kings,California,21951,218.00,6031,CA,County,152940,14352.69,142.54,142.54,14352.69,7,3,35.00,0.00
417381,2021-02-25,Merced,California,28915,395.00,6047,CA,County,277680,10413.07,142.25,142.25,10413.07,8,8,41.00,1.00
46865,2021-02-25,San Joaquin,California,66569,1082.00,6077,CA,County,762148,8734.39,141.97,141.97,8734.39,9,18,107.00,21.00
26630,2021-02-25,Fresno,California,94794,1412.00,6019,CA,County,999101,9487.93,141.33,141.33,9487.93,10,14,53.00,11.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
969739,2021-02-25,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,37,1,7.00,0.00
263318,2021-02-25,Imperial,California,26843,585.00,6025,CA,County,181215,14812.79,322.82,322.82,14812.79,1,2,3.00,1.00
557960,2021-02-25,Kings,California,21951,218.00,6031,CA,County,152940,14352.69,142.54,142.54,14352.69,7,3,35.00,0.00
131368,2021-02-25,San Bernardino,California,285334,2673.00,6071,CA,County,2180085,13088.21,122.61,122.61,13088.21,12,4,0.00,0.00
1992,2021-02-25,Los Angeles,California,1188101,21102.00,6037,CA,County,10039107,11834.73,210.20,210.20,11834.73,2,5,2027.00,115.00
27342,2021-02-25,Riverside,California,289185,3749.00,6065,CA,County,2470546,11705.31,151.75,151.75,11705.31,6,6,225.00,42.00
165069,2021-02-25,Kern,California,102627,811.00,6029,CA,County,900202,11400.44,90.09,90.09,11400.44,25,7,295.00,25.00
417381,2021-02-25,Merced,California,28915,395.00,6047,CA,County,277680,10413.07,142.25,142.25,10413.07,8,8,41.00,1.00
72942,2021-02-25,Tulare,California,47784,746.00,6107,CA,County,466195,10249.79,160.02,160.02,10249.79,5,9,54.00,7.00
58865,2021-02-25,Stanislaus,California,55775,944.00,6099,CA,County,550660,10128.75,171.43,171.43,10128.75,4,10,150.00,3.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
263318,2021-02-25,Imperial,California,26843,585.00,6025,CA,County,181215,14812.79,322.82,322.82,14812.79,1,2,3.00,1.00
1992,2021-02-25,Los Angeles,California,1188101,21102.00,6037,CA,County,10039107,11834.73,210.20,210.20,11834.73,2,5,2027.00,115.00
460417,2021-02-25,Inyo,California,1253,36.00,6027,CA,County,18039,6946.06,199.57,199.57,6946.06,3,28,3.00,1.00
58865,2021-02-25,Stanislaus,California,55775,944.00,6099,CA,County,550660,10128.75,171.43,171.43,10128.75,4,10,150.00,3.00
72942,2021-02-25,Tulare,California,47784,746.00,6107,CA,County,466195,10249.79,160.02,160.02,10249.79,5,9,54.00,7.00
27342,2021-02-25,Riverside,California,289185,3749.00,6065,CA,County,2470546,11705.31,151.75,151.75,11705.31,6,6,225.00,42.00
557960,2021-02-25,Kings,California,21951,218.00,6031,CA,County,152940,14352.69,142.54,142.54,14352.69,7,3,35.00,0.00
417381,2021-02-25,Merced,California,28915,395.00,6047,CA,County,277680,10413.07,142.25,142.25,10413.07,8,8,41.00,1.00
46865,2021-02-25,San Joaquin,California,66569,1082.00,6077,CA,County,762148,8734.39,141.97,141.97,8734.39,9,18,107.00,21.00
26630,2021-02-25,Fresno,California,94794,1412.00,6019,CA,County,999101,9487.93,141.33,141.33,9487.93,10,14,53.00,11.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,322.82,14812.79,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3442,02/21/21,Lassen,5558,19.00,18179.44,62.15,62.15,18320.09,37,1,0.00,0.00
3443,02/22/21,Lassen,5591,19.00,18287.38,62.15,62.15,18320.09,37,1,33.00,0.00
3444,02/23/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,37,1,3.00,0.00
3445,02/24/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,37,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,322.82,14812.79,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,322.82,14812.79,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3516,02/21/21,Fresno,94234,1367.00,9431.88,136.82,141.33,9487.93,10,14,261.00,0.00
3517,02/22/21,Fresno,94374,1367.00,9445.89,136.82,141.33,9487.93,10,14,140.00,0.00
3518,02/23/21,Fresno,94584,1397.00,9466.91,139.83,141.33,9487.93,10,14,210.00,30.00
3519,02/24/21,Fresno,94741,1401.00,9482.62,140.23,141.33,9487.93,10,14,157.00,4.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)